In [6]:
# Libraries
import os
import re
import json
import folium
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geonamescache import GeonamesCache
from urllib.request import urlopen
from helpers.temp_converter import kelvinToCelsius
from helpers.random_cities_generator import getRandomCities


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

api_key = "768d81e0c5c733e2a375488139b78bb0"


# getting lat and lon of random city
def getGeoResponse(city):
    pattern = r"^(?!.*\s)[\x00-\x7F]+$"
    city_name = str(city)
    if re.match(pattern, city_name):
        baseGeoUrl = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=1&appid={api_key}"
        try:
            geoResponse = urlopen(baseGeoUrl)
            return geoResponse
        except Exception as e:
            print(f"error while opening url: {str(e)}")
            return None



def getCitiesCoords(num_cities):
    city_coords = []
    cities = getRandomCities(num_cities)
    for city in cities:
        city_response = getGeoResponse(city)
        if city_response is not None:
            try:
                city_dict = {}
                city_orig = json.loads(city_response.read())
                geo_df = pd.DataFrame(city_orig)
                city_dict["lat"] = geo_df.loc[0, "lat"]
                city_dict["lon"] = geo_df.loc[0, "lon"]
                city_coords.append(city_dict)
            except Exception as e:
                print(f"Error occured reading the response: {str(e)}")

    return city_coords



def getWeatherDataWithCoords(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    # Send a request to the API and retrieve weather data for the coordinates
    response = urlopen(url)
    weather_data = json.loads(response.read())
    # Check if a city name is available in the response
    city_dict = {}
    if "name" in weather_data:
        print(f"the weather: {weather_data}")
        city_dict["city_name"] = weather_data["name"]
        city_dict["lat"] = weather_data["coord"]["lat"]
        city_dict["lon"] = weather_data["coord"]["lon"]
        city_dict["curr_temp"] = kelvinToCelsius(weather_data["main"]["temp"])
        city_dict["max_temp"] = kelvinToCelsius(weather_data["main"]["temp_max"])
        city_dict["min_temp"] = kelvinToCelsius(weather_data["main"]["temp_min"])
        humidity = weather_data["main"]["humidity"]
        city_dict["humidity"] = f"{humidity}%"
        city_dict["sunrise"] = pd.to_datetime(weather_data["sys"]["sunrise"], unit="s")
        city_dict["sunset"] = pd.to_datetime(weather_data["sys"]["sunset"], unit="s")

    return city_dict
    


def getCitiesWeatherData():
    # trial and error mit der anzahl cities, gemäss docs gehen 60 calls/min mit dem free plan
    city_coords = getCitiesCoords(10)
    city_list = []
    for city in city_coords:
        lat = city["lat"]
        lon = city["lon"]
        city_data = getWeatherDataWithCoords(lat=lat, lon=lon) #returns a dict
        if len(city_data) > 0:
            city_list.append(city_data)

    return city_list


cities = getCitiesWeatherData()
df = pd.DataFrame(cities)

df


the weather: {'coord': {'lon': 3.5706, 'lat': 47.7961}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 291.82, 'feels_like': 291.36, 'temp_min': 291.82, 'temp_max': 291.82, 'pressure': 1021, 'humidity': 62, 'sea_level': 1021, 'grnd_level': 1006}, 'visibility': 10000, 'wind': {'speed': 6.49, 'deg': 24, 'gust': 7.71}, 'clouds': {'all': 0}, 'dt': 1684492326, 'sys': {'type': 2, 'id': 2007717, 'country': 'FR', 'sunrise': 1684469012, 'sunset': 1684524064}, 'timezone': 7200, 'id': 3035843, 'name': 'Auxerre', 'cod': 200}
the weather: {'coord': {'lon': -73.7809, 'lat': 45.857}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 278.66, 'feels_like': 277.73, 'temp_min': 275.85, 'temp_max': 282.33, 'pressure': 1022, 'humidity': 75, 'sea_level': 1022, 'grnd_level': 1014}, 'visibility': 10000, 'wind': {'speed': 1.46, 'deg': 195, 'gust': 3.74}, 'clo

,city_name,lat,lon,curr_temp,max_temp,min_temp,humidity,sunrise,sunset
0,Auxerre,47.7961,3.5706,18.67,18.67,18.67,62%,2023-05-19 04:03:32,2023-05-19 19:21:04
1,Saint-Lin-Laurentides,45.8570,-73.7809,5.51,9.18,2.70,75%,2023-05-19 09:19:24,2023-05-20 00:24:02
2,Mathura,27.4643,77.6489,42.00,42.00,42.00,9%,2023-05-18 23:59:08,2023-05-19 13:32:49
3,Ikere-Ekiti,7.4913,5.2322,32.15,32.15,32.15,54%,2023-05-19 05:21:18,2023-05-19 17:50:01
4,Calw,48.7112,8.7452,14.14,15.82,12.78,71%,2023-05-19 03:39:29,2023-05-19 19:03:43
5,Sovetsk,55.0810,21.8883,15.00,15.00,15.00,49%,2023-05-19 02:18:26,2023-05-19 18:39:37
